# 네이버 뉴스 요약
* 입력 : 네이버 뉴스 url, 요약 비율
* 출력 : matrix 혹은 그래프 활용 textrank 구현 이용한 문서 요약

출력 : 요약문(요약비율 적용), 원문

In [12]:
import requests
from bs4 import BeautifulSoup as bs
import re
from konlpy.tag import Komoran
import pandas as pd
import numpy as np
import math
from collections import Counter
from collections import defaultdict

In [13]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=277&aid=0004511484'
resp = requests.get(url)
soup = bs(resp.content)
Text = soup.select('div._article_body_contents')[0].text\
.replace('// flash 오류를 우회하기 위한 함수 추가','')\
.replace('function _flash_removeCallback() {}','').strip()
body = re.sub(r'\[\w{0,10} ?[가-힣]{2,5} ? 기자|\]|\n|\t|\w{0,5} ?기자 ?[a-zA-Z]*@[a-zA-Z]*.*','', Text)


In [14]:
komoran= Komoran()
komoTokens = komoran.pos(body)

In [15]:

for i in range(len(komoTokens)):
    if komoTokens[i][1]=='NNG' and komoTokens[i+1][1] == 'XSN':
        komoTokens[i] = re.sub("\', \'NNG\'\) ?\(\'", "", str(komoTokens[i])+str(komoTokens[i+1]))
        komoTokens[i] = eval(re.sub("XSN", "NNG", komoTokens[i]))
stopPos = re.sub("\(.{1,15}\'(E\w{1,2}|J\w{1,2}|XS\w|SE|SF|SO|SP|SS|SW|VCP)\'\),? ?","",str(komoTokens))
stopwords = re.sub("\(\'(등|들|있)\', \'\w{1,3}\'\),? ?","", stopPos)
onlyText = re.sub("\(|,? ?\'\w{1,3}\'\)","",stopwords)
prodText = eval(onlyText)
voca = list(set(prodText))
voca_len = len(voca)

pd.DataFrame(Counter(prodText).most_common()[:10])[0]

0     글꼴
1     부리
2    화면용
3    사용자
4     개발
5     마루
6     한글
7    네이버
8     새롭
9     설계
Name: 0, dtype: object

In [ ]:
def get_sentence_weight (sentence, keywords):
    sen_list = sentence.split(' ')
    window_start = 0; window_end = -1;
    # 탐색이 아니라 문장 내 중요한 키워드를 계산하기 위한 윈도우
    
    for i in range(len(sen_list)):
        if sen_list[i] in keywords:
            window_start = i
            break
            
    for i in range(len(sen_list) -1, 0, -1):
        if sen_list[i] in keywords:
            window_end = i
            break
    
    if window_start > window_end :
        return 0
    
    window_size = window_end - window_start + 1
    
    keywords_cnt = 0
    for w in sen_list :
        if w in keywords:
            keywords_cnt +=1
    return keywords_cnt*keywords_cnt/window_size

def summarize(content, max_no_of_sentences = 10):
    
    txt = content
    word_list = get_words(txt)
    keywords = get_keywords(word_list, 0.01, 0.5)
    print(keywords)
    
    sentence_list = get_sentences(txt)
    sentence_weight = []
    for sen in sentence_list:
        sentence_weight.append((get_sentence_weight(sen, keywords), sen))
    sentence_weight.sort(reverse=True)
    ret_list = []
    ret_cnt = min(max_no_of_sentences, len(sentence_list))
    
    for i in range(ret_cnt):
        ret_list.append (str(sentence_weight[i][0])+':'+sentence_weight[i][1]+'.')
    return ret_list